In [19]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from joblib import dump

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score


In [20]:
DATA_PATH = Path("data/scas_sf_19item_balanced_with_score.csv")
df = pd.read_csv(DATA_PATH)

print(df.shape)
df.head()


(6000, 22)


,q01,q02,q03,q04,q05,q06,q07,q08,q09,q10,...,q13,q14,q15,q16,q17,q18,q19,anxiety_score,anxiety_level_num,anxiety_level
0,1,0,1,1,0,2,1,1,0,0,...,1,1,0,0,0,0,1,12,0,low
1,1,1,0,2,2,1,0,0,0,0,...,2,0,0,0,0,0,0,11,0,low
2,0,1,0,1,1,0,1,1,0,0,...,0,0,1,1,1,1,0,10,0,low
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,4,0,low
4,0,1,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,6,0,low


In [21]:
FEATURE_COLS = [c for c in df.columns if c.startswith("q")]  # q01..q19
LABEL_COL = "anxiety_score"

X = df[FEATURE_COLS].copy()
y = df[LABEL_COL].astype(float).copy()

X = X.apply(pd.to_numeric, errors="coerce").fillna(0)

print("Features:", len(FEATURE_COLS))
print("Score range:", y.min(), "to", y.max())



Features: 19
Score range: 1.0 to 50.0


In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [23]:
model = Pipeline([
    ("scaler", StandardScaler()),
    ("reg", Ridge(alpha=1.0))
])

model.fit(X_train, y_train)
print("Training done.")


Training done.


In [24]:
pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, pred))
print("R2 :", r2_score(y_test, pred))


MAE: 0.0003484714218948618
R2 : 0.9999999987717761


In [29]:
BACKEND_ML_DIR = Path("../ml").resolve()
BACKEND_ML_DIR.mkdir(parents=True, exist_ok=True)

MODEL_PATH = BACKEND_ML_DIR / "model.joblib"
META_PATH  = BACKEND_ML_DIR / "meta.json"

dump(model, MODEL_PATH)

meta = {
    "label_col": LABEL_COL,
    "feature_cols": FEATURE_COLS,
    "max_score": int(len(FEATURE_COLS) * 3)  # 57
}
META_PATH.write_text(json.dumps(meta, indent=2))

print("Saved model:", MODEL_PATH)
print("Saved meta :", META_PATH)
meta


Saved model: C:\Users\Dinith\OneDrive - Sri Lanka Institute of Information Technology\Desktop\y4s1\resesarch\p1\ml\model.joblib
Saved meta : C:\Users\Dinith\OneDrive - Sri Lanka Institute of Information Technology\Desktop\y4s1\resesarch\p1\ml\meta.json


{'label_col': 'anxiety_score',
 'feature_cols': ['q01',
  'q02',
  'q03',
  'q04',
  'q05',
  'q06',
  'q07',
  'q08',
  'q09',
  'q10',
  'q11',
  'q12',
  'q13',
  'q14',
  'q15',
  'q16',
  'q17',
  'q18',
  'q19'],
 'max_score': 57}